# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [95]:
!cat publications.tsv

url_slug	pub_date	title	category	venue	excerpt	citation	paper_url	slides_url
alignment_analysis	2025-05-15	How does Alignment Enhance LLMs' Multilingual Capabilities? A Language Neurons Perspective	preprint	Preprint	Multilingual Alignment is an effective and representative paradigm to enhance LLMs’ multilingual capabilities, which transfers the capabilities from the high-resource languages to the low-resource languages. Meanwhile, some researches on language-specific neurons reveal that there are language-specific neurons that are selectively activated in LLMs when processing different languages. This provides a new perspective to analyze and understand LLMs’ mechanisms more specifically in multilingual scenarios. In this work, we propose a new finer-grained neuron identification algorithm, which detects language neurons (including language-specific neurons and language-related neurons) and language-agnostic neurons. Furthermore, based on the distributional characteristics of different

## Import pandas

We are using the very handy pandas library for dataframes.

In [96]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [97]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,url_slug,pub_date,title,category,venue,excerpt,citation,paper_url,slides_url
0,alignment_analysis,2025-05-15,How does Alignment Enhance LLMs' Multilingual ...,preprint,Preprint,Multilingual Alignment is an effective and rep...,"Shimao Zhang, Zhejian Lai, Xiang Liu, Shuaijie...",https://arxiv.org/pdf/2505.21505,NaN
1,DCSQE,2025-02-01,Alleviating Distribution Shift in Synthetic Da...,conference,ACL Long Paper,Quality Estimation (QE) models evaluate the qu...,"Xiang Geng, Zhejian Lai, Jiajun Chen, Hao Yang...",https://arxiv.org/pdf/2502.19941,NaN
2,transLLM,2025-02-01,Why Not Transform Chat Large Language Models t...,preprint preprint,Preprint,"Chat large language models (LLMs), fine-tuned ...","Xiang Geng, Ming Zhu, Jiahuan Li, Zhejian Lai,...",https://arxiv.org/pdf/2405.13923,NaN
3,MQMQE,2023-12-01,Unify word-level and span-level tasks: NJUNLP’...,workshop,WMT2023 Shared Task,We introduce the submissions of the NJUNLP tea...,"Xiang Geng, Zhejian Lai, Yu Zhang, Shimin Tao,...",https://arxiv.org/pdf/2309.13230,NaN
4,CBSQE,2023-12-01,Improved pseudo data for machine translation q...,conference,EMNLP Long Paper,Machine translation(MT) quality estimation(QE)...,"Xiang Geng, Yu Zhang, Zhejian Lai, Shuaijie Sh...",https://aclanthology.org/2023.emnlp-main.764.pdf,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [98]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [99]:

import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date[:7]) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date[:7]) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += f"""\ncategory: {item.category}"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
        
    md += "\nbibtexurl: '/files/" + html_filename + ".bib'"
        
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    # if len(str(item.excerpt)) > 5:
    #     md += "\n" + html_escape(item.excerpt) + "\n"

    # if len(str(item.slides_url)) > 5:
    #     md += "\n[Download slides here](" + item.slides_url + ")\n" 

    # if len(str(item.paper_url)) > 5:
    #     md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    # md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [100]:
html_escape(item.excerpt)

'Machine translation(MT) quality estimation(QE) is a crucial task to estimate the quality of MT outputs when reference translations are unavailable. Many studies focus on generating pseudo data using large parallel corpus and achieve remarkable success in the supervised setting. However, pseudo data solutions are less satisfying in unsupervised scenarios because the pseudo labels are inaccurate or the pseudo translations differ from the real ones. To address these problems, we propose to generate pseudo data using the MT model with constrained beam search~(CBSQE). CBSQE preserves the reference parts with high MT probabilities as correct translations, while the rest parts as the wrong ones for MT generation. Therefore, CBSQE can reduce the false negative labels caused by synonyms. Overall, beam search will prefer a more real hypothesis with a higher MT generation likelihood. Extensive experiments demonstrate that CBSQE outperforms strong baselines in both supervised and unsupervised set

In [101]:
!ls ../_publications/

2023-12-CBSQE.md              2025-02-transLLM.md
2023-12-MQMQE.md              2025-05-alignment_analysis.md
2025-02-DCSQE.md


In [102]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
